Valid for ping 0.25 (26/10/2024)

## Data access advanced concepts

In [ ]:
%matplotlib widget

import numpy as np
import themachinethatgoesping as pingmachine
from matplotlib import pyplot as plt
from tqdm.auto import tqdm

### 1. Advanced use of find_files()

In [ ]:
# find_files() searches the input folder recursively (aka, including subfolders)
folder = r'C:\Users\Schimel_Alexandre\Data\MBES\Kongsberg all'
files = pingmachine.echosounders.index_functions.find_files(folder, [".all","wcd"])
print(len(files))

In [ ]:
# find_files() can use a list of folders as input
folders = []
folders.append(r'C:\Users\Schimel_Alexandre\Data\MBES\Kongsberg all\EM302')
folders.append(r'C:\Users\Schimel_Alexandre\Data\MBES\Kongsberg all\EM710')
folders.append(r'C:\Users\Schimel_Alexandre\Data\MBES\Kongsberg all\EM2040')
files = pingmachine.echosounders.index_functions.find_files(folders, [".all","wcd"])
print(len(files))

In [ ]:
# pairs of files (e.g. .all and .wcd) don't have to be in the same folders. 
# Remember they are only paired later, by a File Handler

### 2. Advanced use of File Handler

In [ ]:
# Create a File Handler to access the raw data files
# TO DO - modify to talk about caching
folder = r'C:\Users\Schimel_Alexandre\Data\MBES\Kongsberg all\EM2040\KV-Meritaito_2024_EM2040_Sorvest-F-1-1-extracts-WCD'
files = pingmachine.echosounders.index_functions.find_files(folder, [".all","wcd"])
fileHandler = pingmachine.echosounders.kongsbergall.KongsbergAllFileHandler(files)

### 3. Advanced access of Pings

In [ ]:
# pingcontainer are mostly for fast filtering of pings (eg on basis of sonar head, or location)

In [ ]:
# Access an individual ping in a Ping Container by indexing
pingContainer = fileHandler.get_pings()
ping42 = pingContainer[42]
type(ping42)

In [ ]:
#  This approach only works for a single ping. If you index a Ping Container for multiple pings, you will get a (smaller) Ping Container object
pings0To42 = pingContainer[0:42]
type(pings0To42)

In [ ]:
# If you want a list of Ping objects, you need to loop over each element of the Ping Container
pings0To42 = []
for i in range(42):
    pings0To42.append(pingContainer[i])
print(f"pings0To42 is now a {type(pings0To42)} where each element is a {type(pings0To42[0])}")

In [ ]:
# But you can also directly create lists of pings selected from a Ping Container by filtering for a specific critiera, for example pings containing watercolumn data
pingsWithWC = pingmachine.pingprocessing.filter_pings.by_features(pingContainer,['watercolumn.amplitudes'])
print(f"pingsWithWC is a {type(pingsWithWC)} where each element is a {type(pingsWithWC[0])}")